# Treinamento e Análise

In [1]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
os.listdir('../../01_extract_data/')
# ..= voltar 1 pasta
# . = daqui pra cá

['.ipynb_checkpoints', 'Start_Game_v4.ipynb', 'dados.pkl', 'dados_turno.pkl']

In [3]:
df = pd.read_pickle('../../01_extract_data/dados.pkl')

In [4]:
df.tail()

,1,2,3,4,5,6,7,8,9,resultado
79995,3,8,9,4,1,7,2,5,6,draw
79996,5,3,0,2,4,6,1,0,0,lost
79997,5,1,4,3,8,0,6,2,7,lost
79998,9,8,5,3,6,2,4,7,1,draw
79999,0,0,5,2,1,0,3,0,4,win


In [5]:
# Remover dados duplicados + resertar indice + dropar coluna index
#df = df.drop_duplicates().reset_index().drop('index',axis=1)
df.tail()

,1,2,3,4,5,6,7,8,9,resultado
79995,3,8,9,4,1,7,2,5,6,draw
79996,5,3,0,2,4,6,1,0,0,lost
79997,5,1,4,3,8,0,6,2,7,lost
79998,9,8,5,3,6,2,4,7,1,draw
79999,0,0,5,2,1,0,3,0,4,win


In [6]:
win = 1
draw = 0.5
lost = 0

In [7]:
def resultado_int(x):
    x=x[0]
    if x == 'win':
        return win
    if x == 'lost':
        return lost
    return draw

In [8]:
df['resultado'] = df[['resultado']].apply(resultado_int,axis=1)

# Separação dos dados

In [9]:
x = df.drop('resultado',axis=1)
y = df[['resultado']]


x.shape,y.shape

((80000, 9), (80000, 1))

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size = 0.1,
)

In [12]:
# RESETAR OS índices, para igualar os valores corretos de X e Y
x_train.reset_index().drop('index',axis=1)
y_train.reset_index().drop('index',axis=1)

x_test.reset_index().drop('index',axis=1)
y_test.reset_index().drop('index',axis=1)


# não sei porquê, tive que fazer essa conversão para
# o modelo ser aceito... nunca aconteceu isso antes
x_train = np.asarray(x_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [13]:
import tensorflow as tf

In [14]:
x_train.shape

(72000, 9)

In [15]:
y_train.shape

(72000, 1)

In [16]:
x_train.shape[1]

9

### Create your model

In [17]:
# PODER FAZER LOAD em vez de criar novamente...

model = tf.keras.models.Sequential()

val=9


model.add(tf.keras.layers.Dense(
    units = val*20,
    activation = 'relu',
    input_shape = (x_train.shape[1],)
))


model.add(tf.keras.layers.Dense(
    units = val*20,
    activation = 'relu',
))

#model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(
    units = val*20,
    activation = 'relu',
))




model.add(tf.keras.layers.Dense(
    units = 1,
    activation = 'sigmoid',
))

model.summary()


#model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
#model.compile(loss = 'mean_squared_error', optimizer = 'Adamax' , metrics = ['accuracy'])
#model.compile(loss = 'mean_squared_error', optimizer = 'RMSprop' , metrics = ['accuracy'])
#model.compile(loss = 'poisson', optimizer = 'RMSprop' , metrics = ['accuracy'])
model.compile(loss ='huber_loss' ,optimizer = 'RMSprop' , metrics = ['accuracy'])

#model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
#----------------------------------------------------------------

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 180)               1800      
_________________________________________________________________
dense_1 (Dense)              (None, 180)               32580     
_________________________________________________________________
dense_2 (Dense)              (None, 180)               32580     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 181       
Total params: 67,141
Trainable params: 67,141
Non-trainable params: 0
_________________________________________________________________


In [27]:
"""

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(18, input_dim=9,kernel_initializer='normal', activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(9, kernel_initializer='normal',activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1,kernel_initializer='normal'))

learning_rate = 0.001
momentum = 0.8

# PODEMOS IMPORTAR A FUNÇÃO E ALTERAR
sgd = tf.keras.optimizers.SGD(lr=learning_rate, momentum=momentum,nesterov=False)

model.compile(loss='mean_squared_error', optimizer=sgd)
model.summary()"""

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 18)                180       
_________________________________________________________________
dropout (Dropout)            (None, 18)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 9)                 171       
_________________________________________________________________
dropout_1 (Dropout)          (None, 9)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 10        
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


### Save <span style="color:blue">MODEL</span>

In [18]:
#model.save('../save_model/model.h5')

### LOAD <span style="color:red">MODEL</span> (Pode fazer load em vez de criar)

In [19]:
#model = tf.keras.models.load_model('../save_model/model.h5')

### LOAD <span style="color:red">WEIGHT</span>

In [20]:
#model.load_weights('../save_weight/weight.h5')

### Calcular os pesos da rede (Se fizer o Load, não precisa)

In [28]:
history = model.fit(
    x_train,
    y_train,
    validation_split=0.1,# quantidade de dados que usa para treinar por época (testar)
    epochs = 100,
    verbose = 1,# Não printar
    max_queue_size=10,# fila
    batch_size=None, # Qauntidade de dados que pula
    validation_freq= 1 # Quantidade de Vezes que ele faz a Validação dos dados... exemplo de 10 em 10 épocas
)

history.history.keys()

Epoch 1/100
2025/2025 [==============================] - 4s 2ms/step - loss: 0.2424 - val_loss: 0.1974
Epoch 2/100
2025/2025 [==============================] - 3s 1ms/step - loss: 0.1967 - val_loss: 0.1974
Epoch 3/100
2025/2025 [==============================] - 3s 1ms/step - loss: 0.1963 - val_loss: 0.1975
Epoch 4/100
2025/2025 [==============================] - 3s 1ms/step - loss: 0.1948 - val_loss: 0.1973
Epoch 5/100
2025/2025 [==============================] - 3s 1ms/step - loss: 0.1962 - val_loss: 0.1972
Epoch 6/100
2025/2025 [==============================] - 3s 1ms/step - loss: 0.1952 - val_loss: 0.1971
Epoch 7/100
2025/2025 [==============================] - 3s 1ms/step - loss: 0.1942 - val_loss: 0.1971
Epoch 8/100
2025/2025 [==============================] - 3s 2ms/step - loss: 0.1946 - val_loss: 0.1971
Epoch 9/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1958 - val_loss: 0.1970
Epoch 10/100
2025/2025 [==============================] - 3s 1ms/step - l

Epoch 80/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1076 - val_loss: 0.1209
Epoch 81/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1046 - val_loss: 0.0982
Epoch 82/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1064 - val_loss: 0.0975
Epoch 83/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1056 - val_loss: 0.1238
Epoch 84/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1049 - val_loss: 0.1181
Epoch 85/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1036 - val_loss: 0.1226
Epoch 86/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1034 - val_loss: 0.1108
Epoch 87/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1042 - val_loss: 0.1194
Epoch 88/100
2025/2025 [==============================] - 2s 1ms/step - loss: 0.1032 - val_loss: 0.1214
Epoch 89/100
2025/2025 [==============================] - 2s 1ms

dict_keys(['loss', 'val_loss'])

In [29]:
plt.figure()
plt.plot(history.history['accuracy'],'.r', label = 'acc',alpha = 0.3)
plt.plot(history.history['val_accuracy'], '.b', label = 'acc_val',alpha = 0.3)
plt.legend()
plt.grid()

plt.figure()
plt.plot(history.history['loss'],'.r', label = 'loss',alpha = 0.3)
plt.plot(history.history['val_loss'],'.b', label = 'loss_val',alpha = 0.3)
plt.legend()
plt.grid()

KeyError: 'accuracy'

# Predicts

In [30]:
pred = model.predict(x_test)

In [31]:
pred_df = pd.DataFrame()
pred_df['pred'] = pred[:,0]
pred_df['y'] = y_test

In [32]:
pred_df

,pred,y
0,0.590905,1.0
1,0.831743,1.0
2,0.850202,1.0
3,0.485977,1.0
4,0.513285,0.5
5,0.496395,0.5
6,0.505333,1.0
7,0.500426,1.0
8,0.848164,1.0
9,0.823168,1.0


In [33]:
def f(x):
    val = x[0]
    if val >= 0.79:
        return win
    if 0.40< val and val < 0.79:
        return draw
    if val <= 0.40:
        return lost

In [34]:
pred_df['pred'] = pred_df[['pred']].apply(f,axis=1)

In [35]:
pred_df.count()

pred    8000
y       8000
dtype: int64

In [36]:
# Ver diferença
pred_df[(pred_df['y'] != pred_df['pred']) &(pred_df['y'] == 0)].head(5)

,pred,y
15,0.5,0.0
25,0.5,0.0
31,0.5,0.0
37,0.5,0.0
41,0.5,0.0


In [37]:
pred_df.count()

pred    8000
y       8000
dtype: int64

In [38]:
pred_df[(pred_df['y'] != pred_df['pred']) & (pred_df['y'] == lost)].count()
# O Modelo quase não erra para prever derrotas

pred    715
y       715
dtype: int64

In [39]:
pred_df[(pred_df['y'] != pred_df['pred']) & (pred_df['y'] == win)].count()
# Nem para Vitorias

pred    2458
y       2458
dtype: int64

In [40]:
pred_df[(pred_df['y'] != pred_df['pred']) & (pred_df['y'] == draw)].count()
# O problema é para valores de empate, ele confunde vitoria com empate

pred    0
y       0
dtype: int64

In [41]:
pred_df

,pred,y
0,0.5,1.0
1,1.0,1.0
2,1.0,1.0
3,0.5,1.0
4,0.5,0.5
5,0.5,0.5
6,0.5,1.0
7,0.5,1.0
8,1.0,1.0
9,1.0,1.0


In [42]:
k = 0
total = len(pred_df['pred'])

for pred,y in zip(pred_df['pred'],y_test[:,0]):
    if pred == y:
        k = k+1
p = k/total
print(f"Porcentagem de Acerto {np.around(p*100,2)}%")

Porcentagem de Acerto 60.34%


# Save <span style="color:blue">WEIGHT</span>

In [43]:
#model.save_weights('../save_weight/weight.h5')